In [ ]:
# import statements
import pandas as pd
import numpy as np

# ctrl data

In [ ]:
ctrl.head(2)

In [ ]:
# clean via keeping data before ICD11 and only BMI
ctrl = ctrl[(ctrl['Concept_Name']=='BMI') & (ctrl['Phy.before.ICD11']=='y')]

In [ ]:
# pivot the data to take the mean of the BMI
ctrl['Result_numeric'] = pd.to_numeric(ctrl['Result'])
ctrl = ctrl.pivot_table(
    index='StudyID',
    values='Result_numeric',
    columns='Concept_Name',
    aggfunc='mean'
)
ctrl = ctrl.rename(columns={'BMI': 'mean_BMI'})

In [ ]:
# categorize mean BMI to bins of [0-25] (underweight), [25-30] (healthy), [30-inf] (obese)
bins = [0, 25, 30, np.inf]
labels = [0, 1, 2]

ctrl['mean_BMI_category'] = pd.cut(ctrl['mean_BMI'], bins=bins, labels=labels, right=False)
ctrl

In [ ]:
ctrl.to_csv("../clean_data/control/phy.control.csv", index=True)

# nafl data

In [ ]:
# upload data
nafl = pd.read_csv('../data/NAFLpatients_Jan2025request/Phy_all.use.final.txt', sep='\t', quotechar='"')

# subset to only BMI and before ICD11
nafl = nafl[(nafl['Concept_Name']=='BMI') & (nafl['Phy.before.ICD11']=='y')]

# pivot the data to take the mean of the BMI
nafl['Result_numeric'] = pd.to_numeric(nafl['Result'])
nafl = nafl.pivot_table(
    index='StudyID',
    values='Result_numeric',
    columns='Concept_Name',
    aggfunc='mean'
)
nafl = nafl.rename(columns={'BMI': 'mean_BMI'})

# categorize mean BMI to bins of [0-25] (underweight), [25-30] (healthy), [30-inf] (obese)
bins = [0, 25, 30, np.inf]
labels = [0, 1, 2]

nafl['mean_BMI_category'] = pd.cut(nafl['mean_BMI'], bins=bins, labels=labels, right=False)

In [ ]:
# also acquire the most recent BMI measurement before their NAFL diagnosis to represent each patient
# upload data
nafl1 = pd.read_csv('../data/NAFLpatients_Jan2025request/Phy_all.use.final.txt', sep='\t', quotechar='"')

# subset to only BMI and before ICD11 and nonpositive days from first NAFL measurement
nafl1 = nafl1[(nafl1['Concept_Name']=='BMI') & (nafl1['Phy.before.ICD11']=='y') & (nafl1['Phy.daysfrom_firstNAFL']<=0)]
# group by StudyID and get the index of the maximum "days from NAFL" from those values
idx = nafl1.groupby('StudyID')['Phy.daysfrom_firstNAFL'].idxmax()
nafl1 = nafl1.loc[idx]

In [ ]:
nafl1 = nafl1[['StudyID', 'Result', 'Phy.daysfrom_firstNAFL']] # only keep these cols

In [ ]:
# categorize last BMI to bins of [0-25] (underweight), [25-30] (healthy), [30-inf] (obese)
nafl1['Result_numeric'] = pd.to_numeric(nafl1['Result'])

bins = [0, 25, 30, np.inf]
labels = [0, 1, 2]

nafl1['last_BMI_category'] = pd.cut(nafl1['Result_numeric'], bins=bins, labels=labels, right=False)
nafl1 = nafl1[['StudyID', 'Result_numeric', 'last_BMI_category']]
nafl1.rename(columns={'Result_numeric': 'last_BMI_before_NAFL'})

In [ ]:
merged_nafl = pd.merge(nafl, nafl1, on='StudyID', how='inner')

In [ ]:
merged_nafl = merged_nafl.rename(columns={'Result_numeric': 'last_BMI'})
merged_nafl

In [ ]:
merged_nafl.to_csv("../clean_data/nafl/phy.nafl.csv", index=False)

In [ ]:
all_nan_features = Phy_Control.groupby("Concept_Name")["Result"].apply(lambda x: x.isna().all())

In [ ]:
all_nan_features = all_nan_features[all_nan_features].index.tolist()
all_nan_features

In [ ]:
# now i need to replace all of the NaN values with TRUE to differentiate it from NaNs when we concat later
Phy_Control.loc[Phy_Control["Concept_Name"].isin(all_nan_features), "Result"] = True

In [ ]:
Phy_Control["Concept_Name"].unique()

In [ ]:
# since some columns are denoting a binary question, split those from columns that are numerical
Categorical_Col = Phy_Control.loc[Phy_Control["Result"].isna(), "Concept_Name"].unique()
Numerical_Col = Phy_Control.loc[~Phy_Control["Result"].isna(), "Concept_Name"].unique()

In [ ]:
Phy_Control[Phy_Control["Concept_Name"] == "Albumin/creatinine ratio"].head(2)

In [ ]:
# check if there are duplicates between the two lists
def check_common_elements(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    return set1.intersection(set2)

Common_Col = check_common_elements(Categorical_Col, Numerical_Col)

In [ ]:
Common_Col

In [ ]:
any_remaining_nan_features = Phy_Control.groupby("Concept_Name")["Result"].apply(lambda x: x.isna().any())
any_remaining_nan_features = any_remaining_nan_features[any_remaining_nan_features].index.tolist()
any_remaining_nan_features

In [ ]:
# apply conditional imputing on these features that have some rows that only have NaNs but others that have more data
def is_numeric_feature(group):
    # Try to coerce to numeric and check if any value survives
    return pd.to_numeric(group["Result"], errors='coerce').notna().any()

feature_is_numeric = Phy_Control.groupby("Concept_Name").apply(is_numeric_feature)

In [ ]:
def conditional_impute(group):
    if feature_is_numeric.get(group.name, False):
        # Numeric → fill NaN with median
        result_numeric = pd.to_numeric(group["Result"], errors='coerce')
        median_val = result_numeric.median()
        group["Result"] = result_numeric.fillna(median_val)
    else:
        # Categorical → fill NaN with mode (most frequent string)
        mode_val = group["Result"].mode().iloc[0] if not group["Result"].mode().empty else "Unknown"
        group["Result"] = group["Result"].fillna(mode_val)
    return group

In [ ]:
Phy_Control_Imputed = Phy_Control.groupby("Concept_Name").apply(conditional_impute).reset_index(drop=True)

In [ ]:
post_imp_nan_features = Phy_Control_Imputed.groupby("Concept_Name")["Result"].apply(lambda x: x.isna().any())
post_imp_nan_features = post_imp_nan_features[post_imp_nan_features].index.tolist()
post_imp_nan_features
# should be empty because all should be filled in for!

In [ ]:
# in the Results column, we have numerical values, boolean, and string. 
# for numerical, replace with the min, max, and mean columns.
# for boolean, replace with binary (1 if seen).
# for string, replace with mode of string.

def classify_feature(group):
    # Coerce to numeric
    numeric_values = pd.to_numeric(group["Result"], errors="coerce")
    if numeric_values.notna().all():
        return "numeric"
    elif set(group["Result"].dropna().unique()) <= {"True", "False"}:
        return "boolean"
    else:
        return "categorical"

feature_types = Phy_Control_Imputed.groupby("Concept_Name").apply(classify_feature)

In [ ]:
df = Phy_Control_Imputed
numeric_features = feature_types[feature_types == "numeric"].index.tolist()
df_numeric = df[df["Concept_Name"].isin(numeric_features)].copy()
df_numeric["Result"] = pd.to_numeric(df_numeric["Result"], errors="coerce")

df_numeric_pivot = df_numeric.pivot_table(
    index="StudyID",
    columns="Concept_Name",
    values="Result",
    aggfunc=["mean", "max", "min"]
)

# Flatten column names
df_numeric_pivot.columns = [f"{agg}_{feat}" for agg, feat in df_numeric_pivot.columns]

In [ ]:
boolean_features = feature_types[feature_types == "boolean"].index.tolist()
df_bool = df[df["Concept_Name"].isin(boolean_features)].copy()

# Convert "True"/"False" strings to actual booleans
df_bool["Result"] = df_bool["Result"].map({True: 1, False: 0})

# Use max — if patient ever had "True", result is 1
df_bool_pivot = df_bool.pivot_table(
    index="StudyID",
    columns="Concept_Name",
    values="Result",
    aggfunc="max"
)

df_bool_pivot.columns = [f"{col}" for col in df_bool_pivot.columns]

In [ ]:
def get_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else "Unknown"

categorical_features = feature_types[feature_types == "categorical"].index.tolist()
df_cat = df[df["Concept_Name"].isin(categorical_features)].copy()

df_cat_pivot = df_cat.pivot_table(
    index="StudyID",
    columns="Concept_Name",
    values="Result",
    aggfunc=get_mode
)

df_cat_pivot.columns = [f"{col}_mode" for col in df_cat_pivot.columns]

In [ ]:
final_df = pd.concat([df_numeric_pivot, df_bool_pivot, df_cat_pivot], axis=1).reset_index()

In [ ]:
final_df.head()

In [ ]:
final_df.to_csv('Phy_Control_NoFeaturesDropped.csv', index=False)

In [ ]:
# drop features present in less than 10% of population
missing_fraction = final_df.isna().mean()
keep_cols = missing_fraction[missing_fraction <= 0.90].index.tolist()

In [ ]:
print(f"Dropped {len(keep_cols)} columns with <10% coverage.")
filtered_df = final_df[keep_cols]

In [ ]:
filtered_df.head()